## Cities on Volcanoes 2018
### Code of the analysis and processing of seismic data of the work entitled "_The 2015 hurricane-induced lahars at Volcán de Colima, México: seismic characterization and numeric modeling_".

In [1]:
using PyPlot
using SAC
using DSP
using ExcelReaders

In [2]:
pa = SAC.read("Patricia250.sac");

Info: Data are little-endian; byteswapping


In [3]:
rmean!(pa);
rtrend!(pa);

In [4]:
#Conversion to physical units. V(m/s)= count*1.164153nV/22.8V/(m/s)
pa.t = pa.t*((1.1641e-9)/(22.8));

In [5]:
#Time vector
ti = collect(0:pa.npts-1)*pa.delta;
n = convert(Int32,floor(pa.npts/2)+1);
tp = (DateTime(2015,10,23,21,0,0,1):(Dates.Millisecond(1))*4:DateTime(2015,10,24,4,0,0));

In [6]:
#Frequency vector
ds = 1/(pa.delta*pa.npts);
f = collect(0:pa.npts-1)*ds;

In [7]:
#Movil average function
function MA(x,n)
    if size(x,1)==1
        x = x'
    end
    y = zeros(length(x))
    sx = size(x,2)
    tape = NaN*(zeros(convert(Int,floor(n/2)),sx))
    x1 = [tape;x;tape]
    n1 = n-1
    for ii=1:size(y,1)
        sel = x1[ii+(0:n1),:]
        y[ii]=mean(sel[!isnan.(sel)]);
    end
    return y
end

MA (generic function with 1 method)

In [ ]:
fig = figure(figsize=(14,6))
plot(tp,pa.t*1000,"g")
xlabel("Time[GMT]", fontsize=16)
xticks(fontsize=17)
yticks(fontsize=17)
ylabel("Velocity [mm/s]", fontsize=17)

In [8]:
#envelope
env = abs.(hilbert(pa.t));

In [ ]:
fig = figure(figsize=(14,6))
plot(tp, env,"g")
xlabel("Time [GMT]", fontsize=16)
xticks(fontsize=17)
ylabel("Amp [m^2/s^2]", fontsize=16)
yticks(fontsize=16)

In [9]:
b = ones(15000)*(1/15000);
env_1m = filt(b,1,env);

In [ ]:
fig = figure(figsize=(14,6))
plot(tp,(abs.(env_1m)).*1000,"k")
xlabel("Time GMT", fontsize=17)
xticks(fontsize=17)
ylabel("Amp", fontsize=17)
yticks(fontsize=16)

In [10]:
#Butterworth filter
resp = Bandpass(6,124,fs=250)
desig = Butterworth(4)
fil = filt(digitalfilter(resp,desig),pa.t);

In [11]:
specf = welch_pgram(fil,fs=250)

DSP.Periodograms.Periodogram{Float64,DSP.Util.Frequencies}([1.18383e-16, 2.36766e-16, 2.36766e-16, 2.36766e-16, 2.36766e-16, 2.36766e-16, 2.36766e-16, 2.36766e-16, 2.36766e-16, 2.36766e-16  …  1.58243e-17, 1.58243e-17, 1.58243e-17, 1.58243e-17, 1.58243e-17, 1.58243e-17, 1.58243e-17, 1.58243e-17, 1.58243e-17, 7.91215e-18], [0.0, 0.00031746, 0.000634921, 0.000952381, 0.00126984, 0.0015873, 0.00190476, 0.00222222, 0.00253968, 0.00285714  …  124.997, 124.997, 124.998, 124.998, 124.998, 124.999, 124.999, 124.999, 125.0, 125.0])

In [ ]:
fig = figure(figsize=(14,6))
plot(specf.freq,specf.power,"g")
xlabel("Frequency[Hz]", fontsize=17)
xticks(fontsize=17)
ylabel("PSD", fontsize=17)
yticks(fontsize=17)

In [12]:
av_spec = MA(specf.power,250);

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] !(::BitArray{2}) at ./deprecated.jl:57
 [3] MA(::Array{Float64,1}, ::Int64) at ./In[7]:13
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/marv/.julia/v0.6/IJulia/src/execute_request.jl:180
 [6] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/marv/.julia/v0.6/Compat/src/Compat.jl:332
 [7] eventloop(::ZMQ.Socket) at /home/marv/.julia/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##15#18)() at ./task.jl:335
while loading In[12], in expression starting on line 1


In [ ]:
fig = figure(figsize=(14,6))
plot(specf.freq,av_spec,"g")
xticks(fontsize=17)
xlabel("Frequency [Hz]", fontsize=17)
ylabel("PSD",fontsize=17)
yticks(fontsize=17)

In [ ]:
fig = figure(figsize=(6,8))
plot(av_spec,specf.freq,"g")
xticks(fontsize=17)
axis("tight")
ax=gca()
ax[:yaxis][:set_ticks_position]("right")
xlim(minimum(av_spec)-0.01e-9,maximum(av_spec))
ylim(minimum(specf.freq),maximum(specf.freq))
yticks(fontsize=17)

In [ ]:
fig = figure(figsize=(14,7))
specgram(pa.t,125,250,pad_to=40,noverlap=50);
xlabel("Time [s]", fontsize=17)
xticks(fontsize=17)
ylabel("Frequency [Hz]", fontsize=17)
yticks(fontsize=17)
colorbar()
#PyPlot.savefig("spec.png")

In [ ]:
fig = figure(figsize=(16,8))
subplot(2,2,2)

plot(tp,pa.t*1000,"g")
xlabel("Velocity [mm/s]",fontsize=11)
xticks(fontsize=11)
ylabel("Time GMT",fontsize=10)
yticks(fontsize=10)

subplot(2,2,3)
plot(av_spec.*-1,specf.freq,"g")
#axis("tight")
#ax=gca()
#ax[:spines]["left"][:set_position]("center")
#xlabel("Frequency[Hz]")
#ylabel("PSD")

subplot(2,2,4)
specgram(pa.t,125,250,pad_to=40,noverlap=50);
xlabel("Time [s]")
xticks(fontsize=11)
ylabel("Frequency [Hz]", fontsize=11)
yticks(fontsize=11)
colorbar()

In [13]:
#time for rain
tt = (DateTime(2015,10,23,21,0,0):Dates.Minute(10):DateTime(2015,10,24,4,0,0));

In [14]:
lluvia = readxl("datos_lluvia.xlsx","Datos!F128:F170");
acum = readxl("datos_lluvia.xlsx","Datos!G128:G170");

In [ ]:
fig = figure(figsize=(12,5))
plot(tt,lluvia,"b")
title("Relation rain-signal", fontsize=15)
ax=gca()
xlabel("Time GMT", fontsize=14)
xticks(fontsize=13)
ylabel("mm", fontsize=14)
yticks(fontsize=13)

ax2=ax[:twinx]()
plot(tp,(abs.(env_1m)).*1000,"k")
xticks(fontsize=13)
ylabel("Amp", fontsize=14)
yticks(fontsize=13)

new_position = [0.06;0.06;0.77;0.91]

ax3=ax[:twinx]()
ax3[:spines]["right"][:set_position](("axes",1.12))
plot(tt,acum,"r")
ax3[:set_position](new_position)
ylabel("mm",fontsize=14)
yticks(fontsize=13)